# Bert for Email Spam Detection

As per the paper, we use the simpletransformers library to instantiate our bert model. More information, including other available models can be found here: https://simpletransformers.ai/docs/classification-specifics/

In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from simpletransformers.classification import ClassificationModel, ClassificationArgs

c:\Developer\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load our Dataset

In [2]:
# Load training data
train_df = pd.read_csv("./datasets/train.csv") 
train_df.rename(columns={'spam': 'labels'}, inplace=True)
train_df.head()

,labels,text
0,0,subject institute international finance annual...
1,1,subject mortgage even worst credit zwzm detail...
2,1,subject partnership mr edward moko independenc...
3,1,subject de la part de enfants ama rue de marty...
4,0,subject synfuel option valuation lenny believe...


In [3]:
# Load test data
test_df = pd.read_csv("./datasets/test.csv")  
test_df.rename(columns={'spam': 'labels'}, inplace=True)
test_df.head()

,labels,text
0,1,subject perfect logo charset koi r thinking br...
1,0,subject storage model security stinson added t...
2,1,subject wall street micro news report homeland...
3,1,subject logo stationer website design much lt ...
4,0,subject video conference ross mcintyre vince r...


## Instantiate Model

We set our hyperparameters based on the paper's guidelines

In [4]:
train_args = ClassificationArgs()

train_args.learning_rate = 4e-5
train_args.num_train_epochs = 3
train_args.train_batch_size = 32
train_args.max_seq_length = 300
train_args.optimizer = "AdamW"
train_args.eval_batch_size = 32

In [5]:
# Instantiate the BERT model
model = ClassificationModel(
    "bert",
    "bert-base-cased",
    num_labels=2,  # Binary Classification
    args=train_args,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# Train the model
model.train_model(train_df)
model.save_model(output_dir="./bert_checkpoint")

Epoch 1 of 3:   0%|          | 0/3 [00:05<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 66.00 MiB (GPU 0; 4.00 GiB total capacity; 3.33 GiB already allocated; 0 bytes free; 3.40 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_df)

In [ ]:
# Extract predicted labels and true labels
predictions = result['preds']
true_labels = test_df['labels'] 

# Calculate accuracy and F1 score
accuracy = accuracy_score(true_labels, predictions)
conf_matrix = confusion_matrix(true_labels, predictions)
class_report = classification_report(true_labels, predictions)
f1 = f1_score(true_labels, predictions)

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)